In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden MoViNet video action recognition

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_movinet_action_recognition.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_movinet_action_recognition.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>                                                                                               <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_movinet_action_recognition.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This notebook demonstrates how to use [MoViNet](https://github.com/tensorflow/models/tree/master/official/projects/movinet) for video action recognition in Vertex AI Model Garden.

### Objective

* Train new models
  * Convert input data to training formats
  * Create [hyperparameter tuning jobs](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) to train new models
  * Find and export best models

* Test trained models
  * Upload models to the [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)
  * Run batch predictions

* Clean up resources

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

### Colab Only
Run the following commands for Colab or skip this section if you use Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).
1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for running batch predictions with the fine tuned model.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
import os

from google.cloud import aiplatform

# The GCP project ID for experiments.
PROJECT_ID = ""  # @param {type:"string"}

# Bucket URI with gs:// prefix.
BUCKET_URI = ""  # @param {type:"string"}

# You can choose a region from https://cloud.google.com/about/locations.
# Only regions prefixed by "us", "asia", or "europe" are supported.
REGION = "us-central1"  # @param {type:"string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

! gcloud config set project $PROJECT_ID

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
CHECKPOINT_BUCKET = os.path.join(BUCKET_URI, "ckpt")

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Download config files.
CONFIG_DIR = os.path.join(BUCKET_URI, "config")

### Define constants

In [ ]:
OBJECTIVE = "var"

# Data converter constants.
DATA_CONVERTER_JOB_PREFIX = "data_converter"
DATA_CONVERTER_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/data-converter"
DATA_CONVERTER_MACHINE_TYPE = "n1-highmem-8"
IMAGE_SIZES = {
    "a0": 172,
    "a1": 172,
    "a2": 224,
    "a3": 256,
    "a4": 290,
    "a5": 320,
}

# Training constants.
TRAINING_JOB_PREFIX = "train"
TRAIN_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/movinet-train"
TRAIN_MACHINE_TYPE = "n1-highmem-32"
TRAIN_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAIN_NUM_GPU = 8

# Evaluation constants.
EVALUATION_METRIC = "accuracy"

# Export constants.
EXPORT_JOB_PREFIX = "export"
EXPORT_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/movinet-model-export"
EXPORT_MACHINE_TYPE = "n1-highmem-8"

# Prediction constants.
# You can adjust accelerator types and machine types to get faster predictions.
PREDICTION_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/movinet-serve"
PREDICTION_PORT = 8501
PREDICTION_ACCELERATOR_COUNT = 1
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
PREDICTION_MACHINE_TYPE = "n1-standard-4"
PREDICTION_JOB_PREFIX = "predict"

### Define common helper functions

In [ ]:
import json
from datetime import datetime
from typing import Any

import tensorflow as tf
import yaml


def get_job_name_with_datetime(prefix: str) -> str:
    """Returns a timestamped job name with the given prefix."""
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def get_label_map(label_map_yaml_filepath: str) -> tuple[dict[int, str], int]:
    """Reads label map from a YAML file and returns the label map with the number of classes."""
    with tf.io.gfile.GFile(label_map_yaml_filepath, "rb") as input_file:
        label_map = yaml.safe_load(input_file.read())["label_map"]
    num_classes = max(label_map.keys()) + 1
    return label_map, num_classes


def get_best_trial(
    model_di: str, max_trial_count: int, evaluation_metric: str
) -> tuple[str, Any]:
    """Finds the best trial directory and eval results from a hyperparameter tuning job."""
    best_trial_dir = ""
    best_trial_evaluation_results = {}
    best_performance = -1

    for i in range(max_trial_count):
        current_trial = i + 1
        current_trial_dir = os.path.join(model_dir, "trial_" + str(current_trial))
        current_trial_best_ckpt_dir = os.path.join(current_trial_dir, "best_ckpt")
        current_trial_best_ckpt_evaluation_filepath = os.path.join(
            current_trial_best_ckpt_dir, "info.json"
        )
        with tf.io.gfile.GFile(current_trial_best_ckpt_evaluation_filepath, "rb") as f:
            eval_metric_results = json.load(f)
            current_performance = eval_metric_results[evaluation_metric]
            if current_performance > best_performance:
                best_performance = current_performance
                best_trial_dir = current_trial_dir
                best_trial_evaluation_results = eval_metric_results
    return best_trial_dir, best_trial_evaluation_results


def print_response_instance(json_str: str, label_map: dict[int, str]):
    """Prints summary of a prediction JSON result from the model response."""
    json_obj = json.loads(json_str)
    if "prediction" not in json_obj:
        print("Error:", json_str)
        return
    instance = json_obj["instance"]
    prediction = json_obj["prediction"]
    gcs_uri = instance["content"]
    time_start = instance.get("timeSegmentStart", "0.0s")
    time_end = instance.get("timeSegmentEnd", "Infinity")
    print(f"---------- Predict {gcs_uri}, {time_start} to {time_end}:")
    for predicted in prediction:
        time = predicted["timeSegmentStart"]
        label = label_map[predicted["label"]]
        confidence = predicted["confidence"]
        print(f"At {time}, detected {label} with {confidence} confidence.")


def find_checkpoint_in_dir(checkpoint_dir: str) -> str:
    """Finds a checkpoint path relative to the directory."""
    for root, dirs, files in tf.io.gfile.walk(checkpoint_dir):
        for file in files:
            if file.endswith(".index"):
                return os.path.join(root, os.path.splitext(file)[0])


def upload_checkpoint_to_gcs(checkpoint_url: str) -> str:
    """Uploads a compressed .tar.gz checkpoint at the given URL to Cloud Storage."""
    filename = os.path.basename(checkpoint_url)
    checkpoint_name = filename.replace(".tar.gz", "")
    print("Download checkpoint from", checkpoint_url, "and store to", CHECKPOINT_BUCKET)
    ! wget $checkpoint_url -O $filename
    ! mkdir -p $checkpoint_name
    ! tar -xvzf $filename -C $checkpoint_name

    checkpoint_path = find_checkpoint_in_dir(checkpoint_name)
    checkpoint_path = os.path.relpath(checkpoint_path, checkpoint_name)

    ! gsutil cp -r $checkpoint_name $CHECKPOINT_BUCKET/
    checkpoint_uri = os.path.join(CHECKPOINT_BUCKET, checkpoint_name, checkpoint_path)
    print("Checkpoint uploaded to", checkpoint_uri)
    return checkpoint_uri


def upload_config_to_gcs(url: str) -> str:
    """Uploads a config file at the given URL to Cloud Storage."""
    filename = os.path.basename(url)
    destination = os.path.join(CONFIG_DIR, filename)
    print("Copy", url, "to", destination)
    ! wget "$url" -O "$filename"
    ! gsutil cp "$filename" "$destination"
    return destination

## Train new models
This section shows how to train new models.
1. Convert input data to training formats
2. Create hyperparameter tuning jobs to train new models
3. Find and export best models

If you already trained models, please go to the section `Test Trained models`.

Please select a model:
* `model_id`: MoViNet model variant ID, one of `a0`, `a1`, `a2`, `a3`, `a4`, `a5`. The model with a larger number requires more resources to train, and is expected to have a higher accuracy and latency. Here, we use `a3` for demonstration purpose. **`a0`, `a1`, and `a2` are not recommended for now as we are currently investigating some inference issues with them.**
* `model_mode`: MoViNet model type, either `base` or `stream`. The base model has a slightly higher accuracy, while the streaming model is optimized for streaming and faster CPU inference. See [official MoViNet docs](https://github.com/tensorflow/models/tree/master/official/projects/movinet) for more information.

**Note**: The prediction container only supports base model (non-streaming) for now. If you train a streaming model, you need to download the model and refer to the [MoViNet official guide](https://github.com/tensorflow/models/blob/master/official/projects/movinet/movinet_streaming_model_training_and_inference.ipynb) for running predictions locally.

In [ ]:
model_id = "a3"  # @param ["a0", "a1", "a2", "a3", "a4", "a5"]
model_mode = "base"  # @param ["base", "stream"]
is_stream = model_mode == "stream"
model_name = f"movinet_{model_id}_{model_mode}"
image_size = IMAGE_SIZES[model_id]

if is_stream:
    export_container_args = {
        "conv_type": "2plus1d",
        "se_type": "2plus3d",
        "activation": "hard_swish",
        "gating_activation": "hard_sigmoid",
        "use_positional_encoding": model_id in {"a3", "a4", "a5"},
    }
else:
    export_container_args = {
        "conv_type": "3d",
        "se_type": "3d",
        "activation": "swish",
        "gating_activation": "sigmoid",
        "use_positional_encoding": False,
    }

### Prepare input data for training

Prepare data in the format as described [here](https://cloud.google.com/vertex-ai/docs/video-data/action-recognition/prepare-data), and then convert them to the training formats by running the cell below:

* `input_file_path`: The input file path to the prepared data.
* `input_file_type`: The input file type, such as `csv` or `jsonl`.
* `output_fps`: The sampling rate of the video; Frames per second.
* `num_frames`: Number of frame to sample around keyframe inputs.
* `min_duration_sec`: Minimum duration in seconds for sampling video clips around keyframe inputs. This is for validation purpose - an error will be thrown if there is not enough context around a keyframe.
* `pos_neg_ratio`: Sampling ratio between positive and negative segments. For example, a pos_neg_ratio of 0.5 samples 1 negative instance every 2 positive instances.
* `split_ratio`: Three comma separated floats indicating the proportion of data to split into train/validation/test. They must add up to 1.
* `num_shard`: Three comma separated integers indicating the shards for train/validation/test.

**Note**: For JSONL input, please use `aiplatform.googleapis.com/ml_use` instead of `ml_use` as the JSON key for ML use in `dataItemResourceLabels`. This is to be consistent with other objectives.

In [ ]:
# This job will convert input data as training format, with given split ratios
# and number of shards on train/test/validation.

data_converter_job_name = get_job_name_with_datetime(
    DATA_CONVERTER_JOB_PREFIX + "_" + OBJECTIVE
)

input_file_path = ""  # @param {type:"string"}
input_file_type = "csv"  # @param ["csv", "jsonl"]
output_fps = 10  # @param {type:"integer"}
num_frames = 32  # @param {type:"integer"}
min_duration_sec = 1.0  # @param {type:"number"}
pos_neg_ratio = 1.0  # @param {type:"number"}
split_ratio = "0.8,0.1,0.1"
num_shard = "10,10,10"
data_converter_output_dir = os.path.join(BUCKET_URI, data_converter_job_name)

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": DATA_CONVERTER_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": DATA_CONVERTER_CONTAINER,
            "command": [],
            "args": [
                "--input_file_path=%s" % input_file_path,
                "--input_file_type=%s" % input_file_type,
                "--objective=%s" % OBJECTIVE,
                "--num_shard=%s" % num_shard,
                "--split_ratio=%s" % split_ratio,
                "--output_dir=%s" % data_converter_output_dir,
                "--output_fps=%d" % output_fps,
                "--num_frames=%d" % num_frames,
                "--min_duration_sec=%f" % min_duration_sec,
                "--pos_neg_ratio=%f" % pos_neg_ratio,
                "--output_shape=%d,%d" % (image_size, image_size),
            ],
        },
    }
]

data_converter_custom_job = aiplatform.CustomJob(
    display_name=data_converter_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

data_converter_custom_job.run()

input_train_data_path = os.path.join(data_converter_output_dir, "train.tfrecord*")
input_validation_data_path = os.path.join(data_converter_output_dir, "val.tfrecord*")
label_map_path = os.path.join(data_converter_output_dir, "label_map.yaml")
print("input_train_data_path for training: ", input_train_data_path)
print("input_validation_data_path for training: ", input_validation_data_path)
print("label_map_path for prediction: ", label_map_path)

### Create a Vertex AI custom job with hyperparameter tuning

You use the Vertex AI SDK to create and run the [hyperparameter tuning job](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) with Vertex AI Model Garden training docker images.

#### Define the following specifications

* `worker_pool_specs`: A list of dictionaries specifying the machine type and docker image. This example defines a single node cluster with one `n1-highmem-32` machine with 8 `NVIDIA_TESLA_V100` GPUs.

  **Note**: We recommend using 8 GPUs for MoViNet-A2 and larger. Since loading video data requires a lot of GPU memory, it is recommended to experiment with a small batch size first.
* `parameter_spec`: Dictionary specifying the parameters to optimize. The dictionary key is the string assigned to the command line argument for each hyperparameter in your training application code, and the dictionary value is the parameter specification. The parameter specification includes the type, min/max values, and scale for the hyperparameter.
* `metric_spec`: Dictionary specifying the metric to optimize. The dictionary key is the `hyperparameter_metric_tag` that you set in your training application code, and the value is the optimization goal.

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

# Input train and validation datasets can be found from the section above
# `Prepare input data for training`.
# Or, set prepared datasets paths if already exist.
# input_train_data_path = ""
# input_validation_data_path = ""
# label_map_path = ""

train_job_name = get_job_name_with_datetime(f"{TRAINING_JOB_PREFIX}_{model_name}")
model_dir = os.path.join(BUCKET_URI, train_job_name)
label_map, num_classes = get_label_map(label_map_path)

# Uploads pretained checkpoint to GCS bucket.
init_checkpoint = f"https://storage.googleapis.com/tf_model_garden/vision/movinet/{model_name}_with_backbone.tar.gz"
init_checkpoint = upload_checkpoint_to_gcs(init_checkpoint)

# Uploads config file according to model_id and streaming options.
config_file = f"{model_id}_stream" if is_stream else model_id
config_file = f"https://raw.githubusercontent.com/tensorflow/models/master/official/projects/movinet/configs/yaml/movinet_{config_file}_gpu.yaml"
config_file = upload_config_to_gcs(config_file)

# The parameters here are mainly for demonstration purpose. Please update them
# for better performance.
trainer_args = {
    "experiment": "movinet_kinetics600",
    "config_file": config_file,
    "input_train_data_path": input_train_data_path,
    "input_validation_data_path": input_validation_data_path,
    "init_checkpoint": init_checkpoint,
    "model_dir": model_dir,
    "num_classes": num_classes,
    "global_batch_size": 16,
    "prefetch_buffer_size": 16,
    "shuffle_buffer_size": 32,
    "train_steps": 2000,
}

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAIN_MACHINE_TYPE,
            "accelerator_type": TRAIN_ACCELERATOR_TYPE,
            # Each training job uses TRAIN_NUM_GPU GPUs.
            "accelerator_count": TRAIN_NUM_GPU,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_CONTAINER_URI,
            "args": [
                "--mode=train_and_eval",
                "--params_override=runtime.num_gpus=%d" % TRAIN_NUM_GPU,
            ]
            + ["--{}={}".format(k, v) for k, v in trainer_args.items()],
        },
    }
]

metric_spec = {"model_performance": "maximize"}

# These learning rates might not be optimal for your selected model type; To
# tune learning rates, try hpt.DoubleParameterSpec with more trials.
LEARNING_RATES = [1e-3, 3e-3]
MAX_TRIAL_COUNT = len(LEARNING_RATES)
parameter_spec = {
    "learning_rate": hpt.DiscreteParameterSpec(values=LEARNING_RATES, scale="linear"),
}

print(worker_pool_specs, metric_spec, parameter_spec)

#### Run the hyperparameter tuning job
* `max_trial_count`: Sets an upper bound on the number of trials the service will run. The recommended practice is to start with a smaller number of trials and get a sense of how impactful your chosen hyperparameters are before scaling up.

* `parallel_trial_count`:  If you use parallel trials, the service provisions multiple training processing clusters. The worker pool spec that you specify when creating the job is used for each individual training cluster.  Increasing the number of parallel trials reduces the amount of time the hyperparameter tuning job takes to run; however, it can reduce the effectiveness of the job overall. This is because the default tuning strategy uses results of previous trials to inform the assignment of values in subsequent trials.

* `search_algorithm`: The available search algorithms are grid, random, or default (None). The default option applies Bayesian optimization to search the space of possible hyperparameter values and is the recommended algorithm.

Click on the generated link in the output to see your run in the Cloud Console.

In [ ]:
train_custom_job = aiplatform.CustomJob(
    display_name=train_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

train_hpt_job = aiplatform.HyperparameterTuningJob(
    display_name=train_job_name,
    custom_job=train_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=MAX_TRIAL_COUNT,
    parallel_trial_count=1,
    project=PROJECT_ID,
    search_algorithm=None,
)

train_hpt_job.run()

print("model_dir is:", model_dir)

### Export model in Tensorflow SavedModel format

In [ ]:
# This job will export models from TF checkpoints to TF saved model format.
# model_dir is from the section above.
best_trial_dir, best_trial_evaluation_results = get_best_trial(
    model_dir, MAX_TRIAL_COUNT, EVALUATION_METRIC
)
best_checkpoint_path = find_checkpoint_in_dir(f"{best_trial_dir}/best_ckpt/")
print("best_trial_dir: ", best_trial_dir)
print("best_trial_evaluation_results: ", best_trial_evaluation_results)
print("best_checkpoint: ", best_checkpoint_path)

container_args = {
    "export_path": f"{model_dir}/best_model",
    "model_id": model_id,
    "num_classes": num_classes,
    "causal": is_stream,
    "checkpoint_path": best_checkpoint_path,
    "assert_checkpoint_objects_matched": False,
    **export_container_args,
}

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": EXPORT_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": EXPORT_CONTAINER_URI,
            "args": ["--{}={}".format(k, v) for k, v in container_args.items()],
        },
    }
]

model_export_job_name = get_job_name_with_datetime(EXPORT_JOB_PREFIX + "_" + OBJECTIVE)
model_export_custom_job = aiplatform.CustomJob(
    display_name=model_export_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

model_export_custom_job.run()

print("best model is saved to: ", container_args["export_path"])

## Test trained models
This section shows the way to test with trained models.
1. Upload and deploy models to the [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction)
2. Run batch predictions

**Note:** The prediction container only works with the base model. If you trained a streaming model, download the model from the exported path and refer to the [MoViNet official guide](https://github.com/tensorflow/models/blob/master/official/projects/movinet/movinet_streaming_model_training_and_inference.ipynb) for running predictions locally.

### Upload model to Vertex AI Model Registry

The following cell uploads the trained model to Vertex AI Model Registry. Skip it if you want to run batch predictions on an already uploaded model instead.

#### Configurable environment variables

* `MODEL_PATH`: Cloud Storage URI to the MoViNet model.
* `BATCH_SIZE`: Batch size for inference. Use a larger value to accelerate GPU prediction.
* `NUM_FRAMES`: Number of frames for a single prediction with the model.
* `FPS`: Video sampling frame per second.
* `OVERLAP_FRAMES`: Allowed overlapping frames between consecutive prediction windows. Set a smaller value for faster inference but less accurate.

In [ ]:
serving_env = {
    "MODEL_ID": "tfvision-movinet-var",
    "MODEL_PATH": container_args["export_path"],
    "BATCH_SIZE": 1,
    "NUM_FRAMES": 32,
    "FPS": output_fps,
    "OVERLAP_FRAMES": 24,
    "OBJECTIVE": OBJECTIVE,
    "IMAGE_WIDTH": image_size,
    "IMAGE_HEIGHT": image_size,
}

model = aiplatform.Model.upload(
    display_name=model_name,
    serving_container_image_uri=PREDICTION_CONTAINER_URI,
    serving_container_ports=[PREDICTION_PORT],
    serving_container_predict_route="/predict",
    serving_container_health_route="/ping",
    serving_container_environment_variables=serving_env,
)

model.wait()

print("The uploaded model name is: ", model_name)

Alternatively, uncomment the following cell to use an already uploaded model. Replace the model name string with that of the existing model.

In [ ]:
# model = aiplatform.Model("projects/123456789/locations/us-central1/models/12345678901234567890")

### Run batch predictions

We will now run batch predictions with the trained MoViNet action recognition model with [Vertex AI Batch Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions).

Please prepare an input JSONL file where each line follows [this format](https://cloud.google.com/vertex-ai/docs/video-data/action-recognition/get-predictions?hl=en#input_data_requirements) and store it in a Cloud Storage bucket. The service account should have read access to the buckets containing the trained model and the input data. See [Service accounts overview](https://cloud.google.com/iam/docs/service-account-overview) for more information.

The [Vertex AI Batch Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-batch-predictions) has a default timeout of 10 minutes. Therefore, please make sure the input video clip is around 5 minutes at 5~10 FPS or you may experience a timeout error. To use this model at a larger scale beyond this notebook demontration, you can try one of the following:

- Pull the serving docker image to a VM or a local machine and send prediction requests directly.
- To process more data concurrently, write a custom [DataFlow](https://cloud.google.com/dataflow) pipeline to send prediction requests to the movinet serving container.
- Divide videos into 5-minute clips and run batch prediction with a small batch size.

In [ ]:
# Path to the prediction input JSONL file.
test_jsonl_path = ""  # @param {type:"string"}
# Full service account name with the suffix `gserviceaccount.com`.
batch_predict_service_account = ""  # @param {type:"string"}

predict_job_name = get_job_name_with_datetime(f"{PREDICTION_JOB_PREFIX}_{model_name}")
predict_destination_prefix = os.path.join(STAGING_BUCKET, predict_job_name)

batch_prediction_job = model.batch_predict(
    job_display_name=predict_job_name,
    gcs_source=test_jsonl_path,
    gcs_destination_prefix=predict_destination_prefix,
    machine_type=PREDICTION_MACHINE_TYPE,
    accelerator_count=PREDICTION_ACCELERATOR_COUNT,
    accelerator_type=PREDICTION_ACCELERATOR_TYPE,
    max_replica_count=1,
    service_account=batch_predict_service_account,
)

batch_prediction_job.wait()

print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

You can then read the prediction response JSONL files in the output directory:

In [ ]:
# The label map file was generated from the section above (`Prepare input data for training`).
for file in tf.io.gfile.glob(os.path.join(predict_destination_prefix, "*/*")):
    with tf.io.gfile.GFile(file, "r") as f:
        for line in f:
            print_response_instance(line, label_map)

## Clean up

In [ ]:
# Delete the trained model.
model.delete()
# Delete custom and hpt jobs.
if data_converter_custom_job.list(filter=f'display_name="{data_converter_job_name}"'):
    data_converter_custom_job.delete()
if train_hpt_job.list(filter=f'display_name="{train_job_name}"'):
    train_hpt_job.delete()
if model_export_custom_job.list(filter=f'display_name="{model_export_job_name}"'):
    model_export_custom_job.delete()
if batch_prediction_job.list(filter=f'display_name="{predict_job_name}"'):
    batch_prediction_job.delete()